# Writeup | Advanced Lane Finding
---
[![Udacity](https://s3.amazonaws.com/udacity-sdc/github/shield-carnd.svg)](http://www.udacity.com/drive)

**Abstract — This notebook is the writeup of the Advanced Lane Finding project.** We apply computer vision techniques using ```OpenCV functions``` to the task of lane finding as part of the SELF-DRIVING CAR nanodegree program. The project is broken down into eight steps, which are:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images
* Apply a distortion correction to raw images
* Use color transforms, gradients, etc, to create a thresholded binary image
* Apply a perspective transform to rectify binary image ("birds-eye view")
* Detect lane pixels and fit to find the lane boundary
* Determine the curvature of the lane and vehicle position with respect to center
* Warp the detected lane boundaries back onto the original image
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

## Camera calibration
---
*The code for this part is in **step1.py** file into the **module** folder. Look at the ```camera_calibrate()``` function.* 

We use a video as sources of images. But the images are distorted. We need to calibrate the camera to perform accurate measurements. To that end, we estimate the parameters defining the camera model, which are the camera calibration matrix, ```mtx```, and the distortion coefficients, ```dist```.

First, we take pictures of chessboard shape with the camera to calibrate, then we detect and correct any distortion errors. To do this, I prepare ```object points``` which are points in ( x, y, z ) coordinate space of the chessboard corners in real world. x and y are horizontal and vertical indices of the chessboard corners. We assume the chessboard is a flat plane, then z is always 0.   


```python
# --- camera_calibrate(), lines 27-29 ---
# prepare object points, like (0,0,0) , (1,0,0) , (2,0,0) ..., (7,5,0)
objp = np.zeros((nx*ny, 3), np.float32)
objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)  # x, y coordinates


```

We convert RGB images to grayscale, then we use ```cv2.findChessboardCorners()``` to find and gather corners coordinates of each chessboard.
```python
# --- camera_calibrate(), lines 34-42 ---
# convert frame to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
# if corners are found, add object points, frame points
if ret == True:
    camera['imgpoints'].append(corners)
    camera['objpoints'].append(objp)
```

Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

The follow picture shows the corners coordinates found in each chessboard.   

![fig.0](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_002_writeup_camera%20calibration.PNG)

I then used the output `objpoints` and `imgpoints` to compute the camera calibration matrix ```mtx``` and the distortion coefficients ```dist``` using ```cv2.calibrateCamera()```.   


```python
# --- camera_calibrate(), line 47 ---
reprojection_error, camera['camera_matrix'], camera['coef_distorsion'], rvecs, tvecs =
       cv2.calibrateCamera(camera['objpoints'], camera['imgpoints'], gray.shape[::-1], None, None)
```

## Apply a distortion correction to raw images
---
*The code is in **step1.py** file into the **module** folder. Look at the ```image_undistort()``` function.* 

From here, I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained the result below.   

```python
# --- image_undistort(), line 71 ---
img_undistorted = cv2.undistort(image, output['camera_matrix'], output['coef_distorsion'], None, output['camera_matrix'])
```

![fig.1](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_003_writeup_code_undistort%20images.png) 


## Create a thresholded binary image
---   
*The code is in **step2.py** file into the **module** folder.*

We use a combinaison of various gradients and color thresholds to generate a binary image.   


### Gradient absolute value

We apply the $Sobel_{x}$ and $Sobel_{y}$ operators to each frame using ```cv2.Sobel()```. And then we apply a threshold on it.

```python
# --- sobel_xy(), sobel_abs(), scale(), mask(), gradient_sobel_abs(), lines 17-42 ---
sobel_x     = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3))
sobel_scale = np.uint8(thresh[1] * sobel_x / np.max(sobel_x))
x_binary      = (sobel_scale >= thresh[0]) & (sobel_scale <= thresh[1])
```

### Gradient magnitude
In addition, we can apply the gradient magnitude and a threshold on it.   
Although it was implemented, I have got a better result without it.

```python
# --- gradient_magnitude(), lines 44-51 ---
sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
gradmag = np.uint8(thresh[1] * np.hypot(sobel_x, sobel_y) / np.max(np.hypot(sobel_x, sobel_y)))
result  = (gradmag >= thresh[0]) & (gradmag <= thresh[1])
```

### Gradient direction
Furthermore, we can apply the gradient direction and a threshold on it.   
Although it was implemented, I have got a better result without it.

```python
# --- gradient_direction(), lines 53-59 ---  
sobel_x    = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=15))
sobel_y    = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=15))
absgraddir = np.arctan2(np.absolute(sobel_x), np.absolute(sobel_y))
result     = (absgraddir >= thresh[0]) & (absgraddir <= thresh[1])
```

The following picture shows results of different combinaisons of gradient thresholds.   

![fig.2](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_004_writeup_show%20binaries%20for%20grayscale%20image.png)

### Color transforms

I have tried several channel extractions from various color spaces. And then I apply a threshold on them.   
At the end, I have only kept the Red, Saturation and Value channels.

```python
# --- threshold_color_gray(), threshold_color_rgb(), threshold_color_hls(), threshold_color_hsv(), lines 61-92 ---
rgb       = mpimg.imread('test.jpg')
r_channel = rgb[:, :, 0]
r_binary  = (r_channel >= thresh[0]) & (r_channel <= thresh[1])

hls       = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
s_channel = hls[:, :, 2]
s_binary  = (s_channel >= thresh[0]) & (s_channel <= thresh[1])

hsv       = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
v_channel = hsv[:, :, index]
v_binary  = (v_channel >= thresh[0]) & (v_channel <= thresh[1])
```

The following picture shows results of these various channel extractions before and after the thresholding.

![fig.3](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_007_writeup_%20merge%20of%205%20and%206.png)


### Combinaison of various gradients and  color thresholds

Finally, I have choosen to combine gradient absolute value in both axis and color thresholds with Red, Saturation and Value channels.

```python
# --- combine_threshold(), lines 115-148 ---
result[(x_binary & y_binary) | (r_binary  & s_binary & v_binary)] = 255
```

The table below shows the threshold values that have been used.   

| Item         		      |    Threshold Values   	| 
|:-----------------------:|:-----------------------:| 
| Gradient absolute value | (20, 100)               |     
| Red channel             | (200, 255)              |     
| Saturation channel      | (100, 255)              |     
| Value channel           | (50, 255)               |    

The picture below shows the final result.   

![fig.4](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_008_writeup_show%20combinaison%20of%20binaries.png)



## Perspective transform ("birds-eye view")
---
*The code for this part is in **step3.py** file into the **module** folder.* 

The `image_warp()` function takes as inputs an image and  apart of source coordinates (`x_top = [ 594, 686 ]`).

I chose to hardcode the destination and remaining source points coordinates in the following manner:
```python
# --- image_warp(), lines 18-25 ---
# four source coordinates
src = np.float32([ [x_top[0], 450], [x_top[1], 450], [1045, 665], [262, 665]])
# four desired coordinates
dst = np.float32([ [262, 100], [1045, 100]         , [1045, 665], [262, 665] ])   
```   

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 594, 450      | 262, 100      | 
| 262, 665      | 262, 665      |
| 1045, 665     | 1045, 665     |
| 686, 450      | 1045, 100     |   

We use `cv2.getPerspectiveTransform()` to compute the perspective trasnform matrix, `M`. Then we transforme the source to the destination image using `cv2.warpPerspective()` with the linear interpolation option.   

The following picture shows the result of a perspective transform on straight lane lines.

![fig.5](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_009_writeup_perspective%20transform.png)

## Detect lane pixels and fit to find the lane boundary
---
*The code for this part is in **tracker.py** file into the **module** folder.* 

I use the following general approach to dectecte lines pixels in the first frame :   
1. locate the approximate location of lane lines using peaks of histogram: `find_histopeak()`
2. start with windows at the bottom
3. scan from bottom to top: `search_sliding_window()`
4. find non-zero pixels within each window: `find_nonzero_pixels()`
5. move the window upwards changing its location along the x axis adding +/- marging to the previous values: `search_sliding_window()`
6. fit a second degree polynomial once we have the coordinates of the lane lines: `find_lanes_init()`
7. use this polynomial to plot the lane lines from top to bottom in the warped image: `draw_lane()`

For the next frames, we skip the sliding windows step once we know where the lines are and we search in a margin around the previous line: `find_lanes_next()`

The following picture shows the result of the dectection of the lines pixels.

![fig6](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_010_writeup_find%20lane%20lines.png)

I have implemented some sanity checks, which are:
- xxx
- yyy

## The curvature of the lane and vehicle position with respect to center
---
*The code for this part is in **tracker.py** file into the **module** folder.* 

Previously, I located the lane line pixels, used their x and y pixel positions to fit a second order polynomial curve:   

\begin{equation*}
f(y) = Ay^2 + By + C
\end{equation*}

The radius of curvature at any point x of the function `x=f(y)` is given as follows:   

\begin{equation*}
R_{curve} = \frac{ ( 1 + (2Ay + B)^{2}  )^{3/2}      }{|2A|}
\end{equation*}

I calculated the radius of curvature at the base of the image:
```python
# --- curvature(), lines 155-158 ---
def curvature(height, lane_x):
    x, y = lane_x[:, 0], lane_x[:, 1]
    coeff = np.polyfit(y * y_mx, x * x_mx, 2)
    return ((1 + (2*coeff[0] *height *y_mx + coeff[1])** 2)** 1.5) / np.absolute(2 *coeff[0])
```

The pixel values are converted to meters with the following ratios:
```python
y_mx = 27 / 720  # meters per pixel in y dimension
x_mx = 3.7 / 812 # meters per pixel in x dimension
```

The vehicle position is calculated at the bottom of the frame as the following:
```python
# --- camera_offset(), lines 159-166 ---
def camera_offset(width, x_left, x_right):
    # calculate the offset of the car on the road
    center_lane = (x_left[-1] + x_right[-1]) / 2
    center_diff = (center_lane - width / 2) * x_mx
    side_pos    = 'left'
    if center_diff <= 0:
        side_pos = 'right'
    return np.absolute(center_diff), side_pos
```

## Warp the detected lane boundaries back onto the original image
---
*The code for this part is in **tracker.py** file into the **module** folder.* 

I warped the detected lane boundaries back onto the original image. I also add the output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

```python
# --- draw_lane(), lines 218-250 ---
def draw_lane(image, left_fit, right_fit, Minv):
    # image: width, height
    (width, height) = reversed(image.shape[:2])
    # extract lines
    (x_left, x_right, y_lane), (left_lane, right_lane, inner_lane) = extract_line(image, left_fit, right_fit)
    # draw the lane onto the image_warped blank image
    road, road_bkg     = np.zeros_like(image), np.zeros_like(image)
    cv2.fillPoly(road, np.int32([left_lane]), color=[255, 0, 0])
    cv2.fillPoly(road, np.int32([right_lane]), color=[0, 0, 255])
    cv2.fillPoly(road, np.int32([inner_lane]), color=[0, 255, 0])
    cv2.fillPoly(road_bkg, np.int32([left_lane]), color=[255, 255, 255])
    cv2.fillPoly(road_bkg, np.int32([right_lane]), color=[255, 255, 255])
    road_warped = cv2.warpPerspective(road, Minv, (width, height), flags=cv2.INTER_LINEAR)
    road_warped_bkg=cv2.warpPerspective(road_bkg, Minv,(width, height), flags=cv2.INTER_LINEAR)
    base = cv2.addWeighted(image, 1.0, road_warped_bkg, -1.0, 0.0)  # 1.3, 0.0)
    result = cv2.addWeighted(base, 1.0, road_warped, 0.7, 0.0)  # 1.3, 0.0)
    # calculate the offset of the car on the road
    center_diff, side_pos = camera_offset(width, x_left, x_right)
    # draw the text showing curvature, offset, and speed
    curverad = self.curvature(height, left_lane)
    cv2.putText(result, 'Radius of Curvature = ' + str(int(curverad))+ ' m', (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, 'Vehicle is ' + str(abs(round(center_diff, 3))) + ' m ' + side_pos 
                + ' of center', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return result
```

The following picture shows the result.

![fig.7](https://raw.githubusercontent.com/chatmoon/SDC_PRJ04_Lane_Finding/master/_1_wip/output_images/_011_writeup_warp%20back.png)

Find here the video output.
[![video0](https://img.youtube.com/vi/iWqL5C8fFyM/0.jpg)](https://www.youtube.com/watch?v=iWqL5C8fFyM)